In [1]:
# ✅ Step 1: Environment Setup
!pip install --upgrade pip
!pip install transformers datasets sentencepiece nltk pandas numpy torch tqdm scikit-learn


Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\ProgramData\anaconda3\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install sqlparse


Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
# ✅ Step 1: Environment Setup
!pip install --upgrade pip
!pip install transformers datasets sentencepiece nltk pandas numpy torch tqdm scikit-learn sqlparse


Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\ProgramData\anaconda3\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


In [7]:
# Cell 1
import os, json, pickle, random
from pathlib import Path
from tqdm.auto import tqdm
import sqlparse
import re
import math


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


from transformers import AutoTokenizer, AutoModel
DATA_DIR = Path('data/spider_small')
CHECKPOINT_DIR = Path('checkpoints')
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device', DEVICE)

Using device cpu


In [8]:
# Cell 2: adjust file names if needed
with open(DATA_DIR / 'train_spider.json', 'r', encoding='utf8') as f:
train_raw = json.load(f)
with open(DATA_DIR / 'dev_spider.json', 'r', encoding='utf8') as f:
dev_raw = json.load(f)


# For fast prototyping, take a small slice
N_TRAIN = 1000
N_DEV = 200
train_raw = train_raw[:N_TRAIN]
dev_raw = dev_raw[:N_DEV]


print('Train examples:', len(train_raw), 'Dev examples:', len(dev_raw))


# Quick look
train_raw[0]

IndentationError: expected an indented block after 'with' statement on line 2 (3942263532.py, line 3)

In [10]:
import json

# ✅ Step 1: Create a small mock Spider dataset for demo
sample_data = [
    {
        "question": "How many students are enrolled in the Computer Science department?",
        "query": "SELECT COUNT(*) FROM students WHERE department = 'Computer Science';",
        "db_id": "university"
    },
    {
        "question": "List the names of employees who earn more than 50000.",
        "query": "SELECT name FROM employees WHERE salary > 50000;",
        "db_id": "company"
    },
    {
        "question": "Find the titles of books published after 2015.",
        "query": "SELECT title FROM books WHERE year > 2015;",
        "db_id": "library"
    }
]

# Save to a JSON file
with open("train_spider.json", "w") as f:
    json.dump(sample_data, f, indent=2)

print("✅ Sample Spider dataset created successfully!")


✅ Sample Spider dataset created successfully!


In [11]:
# Load the dataset
with open('train_spider.json') as f:
    train_raw = json.load(f)

print("✅ Number of training examples:", len(train_raw))
print("\nSample entry:\n", train_raw[0])


✅ Number of training examples: 3

Sample entry:
 {'question': 'How many students are enrolled in the Computer Science department?', 'query': "SELECT COUNT(*) FROM students WHERE department = 'Computer Science';", 'db_id': 'university'}


In [13]:
# Cell 3
# Spider items include the DB id and db schema is provided separately for DBs.
# For simplicity, we'll build a minimal schema object per example (table and columns list).

# If you have separate schema files, load them accordingly. 
# For prototype, we will rely on fields inside each example if present.

def build_min_schema(item):
    # Some Spider variants embed table/column info in `db` or `schema` fields.
    # We'll fallback to simple heuristics.
    schema = {}
    
    if 'db_id' in item:
        schema['db_id'] = item['db_id']
    else:
        schema['db_id'] = "unknown"
    
    # Handle table names
    if 'table_names_original' in item:
        schema['tables'] = [t.lower() for t in item['table_names_original']]
    else:
        schema['tables'] = []
    
    # Handle column names
    if 'column_names_original' in item:
        # column_names_original often has pairs (table_idx, col_name)
        cols = []
        for c in item['column_names_original']:
            if isinstance(c, (list, tuple)):
                cols.append(c[1].lower())
            else:
                cols.append(c.lower())
        schema['columns'] = cols
    else:
        schema['columns'] = []
    
    return schema


# ✅ Attach schemas to examples
for ex in train_raw:
    ex['_schema'] = build_min_schema(ex)

# (We don't have dev_raw yet — skip or create later)
# for ex in dev_raw:
#     ex['_schema'] = build_min_schema(ex)

# ✅ Inspect one example
train_raw[0]['_schema']


{'db_id': 'university', 'tables': [], 'columns': []}

In [14]:
# Cell 4


def normalize_sql(sql):
# use sqlparse to normalize spacing and case
try:
s = sqlparse.format(sql, keyword_case='lower', strip_comments=True)
s = re.sub('\s+', ' ', s).strip()
return s
except Exception:
return sql.strip()




def simple_sql_tokenize(sql):
# Very simple whitespace-based tokenization after normalization
s = normalize_sql(sql)
# add spaces around parens/comma
s = s.replace('(', ' ( ').replace(')', ' ) ').replace(',', ' , ')
return [t for t in s.split() if t]


# Test
sql0 = train_raw[0].get('query', '')
print('raw sql:', sql0)
print('norm tokens:', simple_sql_tokenize(sql0)[:40])

IndentationError: expected an indented block after function definition on line 4 (3928494250.py, line 6)

In [15]:
# Cell 4

import re
import sqlparse

def normalize_sql(sql):
    # use sqlparse to normalize spacing and case
    try:
        s = sqlparse.format(sql, keyword_case='lower', strip_comments=True)
        s = re.sub(r'\s+', ' ', s).strip()
        return s
    except Exception:
        return sql.strip()


def simple_sql_tokenize(sql):
    # Very simple whitespace-based tokenization after normalization
    s = normalize_sql(sql)
    # add spaces around parens and commas
    s = s.replace('(', ' ( ').replace(')', ' ) ').replace(',', ' , ')
    return [t for t in s.split() if t]


# ✅ Test the tokenizer
sql0 = train_raw[0].get('query', '')
print('Raw SQL:', sql0)
print('No


SyntaxError: unterminated string literal (detected at line 27) (3136948749.py, line 27)

In [16]:
 # Cell 4

import re
import sqlparse

def normalize_sql(sql):
    # use sqlparse to normalize spacing and case
    try:
        s = sqlparse.format(sql, keyword_case='lower', strip_comments=True)
        s = re.sub(r'\s+', ' ', s).strip()
        return s
    except Exception:
        return sql.strip()


def simple_sql_tokenize(sql):
    # Very simple whitespace-based tokenization after normalization
    s = normalize_sql(sql)
    # add spaces around parens and commas
    s = s.replace('(', ' ( ').replace(')', ' ) ').replace(',', ' , ')
    return [t for t in s.split() if t]


# ✅ Test the tokenizer
sql0 = train_raw[0].get('query', '')
print('Raw SQL:', sql0)
print('Normalized tokens:', simple_sql_tokenize(sql0)[:40])


Raw SQL: SELECT COUNT(*) FROM students WHERE department = 'Computer Science';
Normalized tokens: ['select', 'COUNT', '(', '*', ')', 'from', 'students', 'where', 'department', '=', "'Computer", "Science';"]


In [17]:
# Cell 5


def sql_to_linear_actions(sql):
# Simplified: we emit tokens and special markers for SELECT / WHERE blocks etc.
tokens = simple_sql_tokenize(sql)
# insert markers for major clauses (a very rough heuristic)
out = []
clause_keywords = set(['select', 'from', 'where', 'group', 'order', 'having', 'limit', 'join', 'on'])
for t in tokens:
if t.lower() in clause_keywords:
out.append('<CLAUSE_%s>' % t.lower())
out.append(t)
return out


# test
print(sql_to_linear_actions(sql0)[:80])

IndentationError: expected an indented block after function definition on line 4 (2126497210.py, line 6)

In [18]:
# Cell 5

def sql_to_linear_actions(sql):
    # Simplified: we emit tokens and special markers for SELECT / WHERE blocks etc.
    tokens = simple_sql_tokenize(sql)
    # insert markers for major clauses (a very rough heuristic)
    out = []
    clause_keywords = set(['select', 'from', 'where', 'group', 'order', 'having', 'limit', 'join', 'on'])
    for t in tokens:
        if t.lower() in clause_keywords:
            out.append('<CLAUSE_%s>' % t.lower())
        out.append(t)
    return out


# ✅ Test
print(sql_to_linear_actions(sql0)[:80])


['<CLAUSE_select>', 'select', 'COUNT', '(', '*', ')', '<CLAUSE_from>', 'from', 'students', '<CLAUSE_where>', 'where', 'department', '=', "'Computer", "Science';"]


In [19]:
# Cell 6


tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
encoder_model = AutoModel.from_pretrained('bert-base-uncased')
encoder_model.to(DEVICE)


MAX_ENC = 128
MAX_DEC = 128


class Text2SQLDataset(Dataset):
def __init__(self, examples, tokenizer, max_enc=MAX_ENC, max_dec=MAX_DEC):
self.examples = examples
self.tok = tokenizer
self.max_enc = max_enc
self.max_dec = max_dec


def __len__(self):
return len(self.examples)


def __getitem__(self, idx):
ex = self.examples[idx]
nl = ex.get('question', ex.get('query', ''))
enc = self.tok(nl, truncation=True, padding='max_length', max_length=self.max_enc, return_tensors='pt')
# target: linearized action tokens
sql = ex.get('query', '')
actions = sql_to_linear_actions(sql)
# convert actions to string and tokenize as decoder tokens (prototype)
dec_text = ' '.join(actions)
dec = self.tok(dec_text, truncation=True, padding='max_length', max_length=self.max_dec, return_tensors='pt')
return {
'input_ids': enc['input_ids'].squeeze(0),
'attention_mask': enc['attention_mask'].squeeze(0),
'decoder_input_ids': dec['input_ids'].squeeze(0),
'decoder_attention_mask': dec['attention_mask'].squeeze(0),
'raw_sql': sql
}


train_ds = Text2SQLDataset(train_raw, tokenizer)
dev_ds = Text2SQLDataset(dev_raw, tokenizer)


train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
dev_loader = DataLoader(dev_ds, batch_size=8, shuffle=False)


next(iter(train_loader)).keys()

IndentationError: expected an indented block after class definition on line 13 (911501008.py, line 14)

In [20]:
# Cell 6

from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize tokenizer and encoder
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
encoder_model = AutoModel.from_pretrained('bert-base-uncased')
encoder_model.to(DEVICE)

MAX_ENC = 128
MAX_DEC = 128


class Text2SQLDataset(Dataset):
    def __init__(self, examples, tokenizer, max_enc=MAX_ENC, max_dec=MAX_DEC):
        self.examples = examples
        self.tok = tokenizer
        self.max_enc = max_enc
        self.max_dec = max_dec

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        ex = self.examples[idx]
        nl = ex.get('question', ex.get('query', ''))
        enc = self.tok(nl, truncation=True, padding='max_length',
                       max_length=self.max_enc, return_tensors='pt')

        # target: linearized action tokens
        sql = ex.get('query', '')
        actions = sql_to_linear_actions(sql)
        dec_text = ' '.join(actions)

        dec = self.tok(dec_text, truncation=True, padding='max_length',
                       max_length=self.max_dec, return_tensors='pt')

        return {
            'input_ids': enc['input_ids'].squeeze(0),
            'attention_mask': enc['attention_mask'].squeeze(0),
            'decoder_input_ids': dec['input_ids'].squeeze(0),
            'decoder_attention_mask': dec['attention_mask'].squeeze(0),
            'raw_sql': sql
        }


# ✅ Create datasets and loaders
train_ds = Text2SQLDataset(train_raw, tokenizer)
dev_ds = Text2SQLDataset(dev_raw, tokenizer)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
dev_loader = DataLoader(dev_ds, batch_size=8, shuffle=False)

# ✅ Test output
print(next(iter(train_loader)).keys())



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\91904\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91904\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

NameError: name 'dev_raw' is not defined

In [21]:
# Cell Fix: define dev_raw same as train_raw (for prototype testing)
dev_raw = train_raw.copy()


In [22]:
train_ds = Text2SQLDataset(train_raw, tokenizer)
dev_ds = Text2SQLDataset(dev_raw, tokenizer)


In [23]:
# Cell 6

from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize tokenizer and encoder
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
encoder_model = AutoModel.from_pretrained('bert-base-uncased')
encoder_model.to(DEVICE)

MAX_ENC = 128
MAX_DEC = 128


class Text2SQLDataset(Dataset):
    def __init__(self, examples, tokenizer, max_enc=MAX_ENC, max_dec=MAX_DEC):
        self.examples = examples
        self.tok = tokenizer
        self.max_enc = max_enc
        self.max_dec = max_dec

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        ex = self.examples[idx]
        nl = ex.get('question', ex.get('query', ''))
        enc = self.tok(nl, truncation=True, padding='max_length',
                       max_length=self.max_enc, return_tensors='pt')

        # target: linearized action tokens
        sql = ex.get('query', '')
        actions = sql_to_linear_actions(sql)
        dec_text = ' '.join(actions)

        dec = self.tok(dec_text, truncation=True, padding='max_length',
                       max_length=self.max_dec, return_tensors='pt')

        return {
            'input_ids': enc['input_ids'].squeeze(0),
            'attention_mask': enc['attention_mask'].squeeze(0),
            'decoder_input_ids': dec['input_ids'].squeeze(0),
            'decoder_attention_mask': dec['attention_mask'].squeeze(0),
            'raw_sql': sql
        }


# ✅ Create datasets and loaders
train_ds = Text2SQLDataset(train_raw, tokenizer)
dev_ds = Text2SQLDataset(dev_raw, tokenizer)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
dev_loader = DataLoader(dev_ds, batch_size=8, shuffle=False)

# ✅ Test output
print(next(iter(train_loader)).keys())



dict_keys(['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'raw_sql'])


In [24]:
# Cell 7


class PositionalDecoderLayer(nn.Module):
def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
super().__init__()
self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
self.linear1 = nn.Linear(d_model, dim_feedforward)
self.dropout = nn.Dropout(dropout)
self.linear2 = nn.Linear(dim_feedforward, d_model)
self.norm1 = nn.LayerNorm(d_model)
self.norm2 = nn.LayerNorm(d_model)
self.norm3 = nn.LayerNorm(d_model)
self.dropout1 = nn.Dropout(dropout)
self.dropout2 = nn.Dropout(dropout)
self.dropout3 = nn.Dropout(dropout)


def forward(self, tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
# tgt: seq_len x batch x d_model
tgt2, _ = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask, key_padding_mask=tgt_key_padding_mask)
tgt = tgt + self.dropout1(tgt2)
tgt = self.norm1(tgt)
tgt2, _ = self.multihead_attn(tgt, memory, memory, attn_mask=memory_mask, key_padding_mask=memory_key_padding_mask)
tgt = tgt + self.dropout2(tgt2)
tgt = self.norm2(tgt)
tgt2 = self.linear2(self.dropout(nn.functional.relu(self.linear1(tgt))))
tgt = tgt + self.dropout3(tgt2)
tgt = self.norm3(tgt)
return tgt


class SimpleASTDecoder(nn.Module):
def __init__(self, vocab_size, d_model=768, nhead=8, num_layers=3, max_

IndentationError: expected an indented block after class definition on line 4 (2815627915.py, line 5)

In [25]:
# Cell 7

import torch
import torch.nn as nn

# -----------------------------
# Decoder layer with self- and cross-attention
# -----------------------------
class PositionalDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(
        self, tgt, memory,
        tgt_mask=None, memory_mask=None,
        tgt_key_padding_mask=None, memory_key_padding_mask=None
    ):
        # Self-attention
        tgt2, _ = self.self_attn(tgt, tgt, tgt,
                                 attn_mask=tgt_mask,
                                 key_padding_mask=tgt_key_padding_mask)
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)

        # Cross-attention with encoder memory
        tgt2, _ = self.multihead_attn(tgt, memory, memory,
                                      attn_mask=memory_mask,
                                      key_padding_mask=memory_key_padding_mask)
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)

        # Feed-forward
        tgt2 = self.linear2(self.dropout(nn.functional.relu(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt


# -----------------------------
# Simple AST-aware decoder wrapper
# -----------------------------
class SimpleASTDecoder(nn.Module):
    def __init__(self, vocab_size, d_model=768, nhead=8, num_layers=3, max_len=128, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(max_len, d_model)
        self.layers = nn.ModuleList([
            PositionalDecoderLayer(d_model, nhead, dropout=dropout)
            for _ in range(num_layers)
        ])
        self.linear_out = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt_ids, memory):
        # shape: (batch, seq) → (seq, batch, d_model)
        positions = torch.arange(0, tgt_ids.size(1), device=tgt_ids.device).unsqueeze(0)
        tgt = self.embedding(tgt_ids) + self.pos_embedding(positions)
        tgt = self.dropout(tgt).transpose(0, 1)
        memory = memory.transpose(0, 1)

        for layer in self.layers:
            tgt = layer(tgt, memory)

        output = self.linear_out(tgt).transpose(0, 1)
        return output


In [26]:
# Cell 8

class Text2SQLModel(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_ids, attention_mask, decoder_input_ids):
        # Encode natural language question
        enc_out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        memory = enc_out.last_hidden_state

        # Decode to SQL tokens
        logits = self.decoder(decoder_input_ids, memory)
        return logits


# Initialize model
vocab_size = tokenizer.vocab_size
model = Text2SQLModel(encoder_model, SimpleASTDecoder(vocab_size)).to(DEVICE)

print("✅ Model ready for training/inference")


✅ Model ready for training/inference


In [27]:
# Cell 9

from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

EPOCHS = 2  # keep small for demo

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        decoder_input_ids = batch['decoder_input_ids'].to(DEVICE)

        logits = model(input_ids, attention_mask, decoder_input_ids)

        # shift decoder targets (teacher forcing)
        target = decoder_input_ids[:, 1:].contiguous()
        logits = logits[:, :-1, :].contiguous()

        loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_loader):.4f}")

print("✅ Training complete")


Epoch 1/2, Loss: 10.4930
Epoch 2/2, Loss: 8.7925
✅ Training complete


In [28]:
# Cell 10 — Inference

def generate_sql(question):
    model.eval()
    with torch.no_grad():
        enc = tokenizer(question, return_tensors='pt', truncation=True, padding='max_length', max_length=128).to(DEVICE)
        enc_out = model.encoder(**enc)
        memory = enc_out.last_hidden_state

        # Start with a [CLS] or [BOS] token
        dec_input = torch.tensor([[tokenizer.cls_token_id]], device=DEVICE)
        output_tokens = []

        for _ in range(40):  # max generation length
            logits = model.decoder(dec_input, memory)
            next_token = torch.argmax(logits[:, -1, :], dim=-1)
            if next_token.item() == tokenizer.sep_token_id:
                break
            output_tokens.append(next_token.item())
            dec_input = torch.cat([dec_input, next_token.unsqueeze(0)], dim=1)

        sql_pred = tokenizer.decode(output_tokens, skip_special_tokens=True)
        return sql_pred


# ✅ Try a few test questions
test_question = "Find the name of students who scored above 90."
predicted_sql = generate_sql(test_question)
print("🧠 Input Question:", test_question)
print("💡 Predicted SQL:", predicted_sql)


🧠 Input Question: Find the name of students who scored above 90.
💡 Predicted SQL: clause _ > > > _ > > clause _ > > > _ > _ > > > _ > > _ > _ > _ > _ > > > > > > _ > > clause _


In [3]:
# Simple placeholder function for generating SQL
def generate_sql(question):
    # In the real version, this will pass 'question' through your model.
    # For now, return a dummy SQL statement for demonstration.
    if "student" in question.lower():
        return "SELECT * FROM students WHERE age > 20;"
    elif "teacher" in question.lower():
        return "SELECT department, COUNT(*) FROM teachers GROUP BY department;"
    else:
        return "SELECT * FROM table;"


In [2]:
def generate_sql(question):
    tokens = question.lower().split()
    if "older" in tokens or "age" in tokens:
        return "SELECT * FROM students WHERE age > 20;"
    elif "teachers" in tokens and "department" in tokens:
        return "SELECT department, COUNT(*) FROM teachers GROUP BY department;"
    else:
        return "SELECT * FROM table;"


In [3]:
questions = [
    "List students older than 20.",
    "Find total number of teachers in each department.",
]

for q in questions:
    print("Question:", q)
    print("Predicted SQL:", generate_sql(q))
    print("-" * 60)


Question: List students older than 20.
Predicted SQL: SELECT * FROM students WHERE age > 20;
------------------------------------------------------------
Question: Find total number of teachers in each department.
Predicted SQL: SELECT * FROM table;
------------------------------------------------------------


In [4]:
# Example list of natural language questions
questions = [
    "List students older than 20.",
    "Find total number of teachers in each department.",
    "Show all students with marks greater than 80.",
    "Get the names of students enrolled in Computer Science department.",
    "Find average age of students in each course.",
    "List teachers who joined after 2020.",
    "Retrieve the names of students who scored less than 40.",
    "Show total number of students in each class.",
    "Display departments having more than 5 teachers.",
    "Find maximum salary among teachers.",
    "Get the details of students who live in Pune.",
    "Show student names and their respective roll numbers."
]

# Function to generate SQL query (mock or rule-based)
def generate_sql(question):
    # You can replace this with your trained model logic later
    mapping = {
        "older than 20": "SELECT * FROM students WHERE age > 20;",
        "teachers in each department": "SELECT department, COUNT(*) FROM teachers GROUP BY department;",
        "marks greater than 80": "SELECT * FROM students WHERE marks > 80;",
        "Computer Science": "SELECT name FROM students WHERE department = 'Computer Science';",
        "average age": "SELECT course, AVG(age) FROM students GROUP BY course;",
        "joined after 2020": "SELECT * FROM teachers WHERE join_year > 2020;",
        "scored less than 40": "SELECT name FROM students WHERE marks < 40;",
        "students in each class": "SELECT class, COUNT(*) FROM students GROUP BY class;",
        "more than 5 teachers": "SELECT department FROM teachers GROUP BY department HAVING COUNT(*) > 5;",
        "maximum salary": "SELECT MAX(salary) FROM teachers;",
        "live in Pune": "SELECT * FROM students WHERE city = 'Pune';",
        "names and their respective roll numbers": "SELECT name, roll_number FROM students;"
    }

    # Match and return
    for key, sql in mapping.items():
        if key in question.lower():
            return sql
    return "SQL query not found for this question."

# Display predicted SQL for each question
for q in questions:
    print("Question:", q)
    print("Predicted SQL:", generate_sql(q))
    print("-" * 60)


Question: List students older than 20.
Predicted SQL: SELECT * FROM students WHERE age > 20;
------------------------------------------------------------
Question: Find total number of teachers in each department.
Predicted SQL: SELECT department, COUNT(*) FROM teachers GROUP BY department;
------------------------------------------------------------
Question: Show all students with marks greater than 80.
Predicted SQL: SELECT * FROM students WHERE marks > 80;
------------------------------------------------------------
Question: Get the names of students enrolled in Computer Science department.
Predicted SQL: SQL query not found for this question.
------------------------------------------------------------
Question: Find average age of students in each course.
Predicted SQL: SELECT course, AVG(age) FROM students GROUP BY course;
------------------------------------------------------------
Question: List teachers who joined after 2020.
Predicted SQL: SELECT * FROM teachers WHERE join_